In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dangjin_fcst = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/당진/dangjin_fcst_data.csv')
dangjin_fcst

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,52.0,7.0,40.0,3.2,187.0,1.0
162204,2021-03-01 08:00:00,55.0,8.0,40.0,4.5,217.0,1.0
162205,2021-03-01 08:00:00,58.0,5.0,55.0,2.2,210.0,1.0
162206,2021-03-01 08:00:00,61.0,1.0,80.0,1.9,164.0,1.0


In [ ]:
dangjin_obs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/당진/dangjin_obs_data.csv')
dangjin_obs

,지점,지점명,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
0,129,서산,2018-03-01 00:00,3.1,3.6,340.0,96.0,NaN
1,129,서산,2018-03-01 01:00,2.8,0.7,140.0,97.0,NaN
2,129,서산,2018-03-01 02:00,2.6,3.2,320.0,95.0,NaN
3,129,서산,2018-03-01 03:00,2.0,1.9,230.0,97.0,NaN
4,129,서산,2018-03-01 04:00,2.2,2.1,180.0,97.0,NaN
...,...,...,...,...,...,...,...,...
25621,129,서산,2021-01-31 19:00,6.7,1.5,200.0,75.0,8.0
25622,129,서산,2021-01-31 20:00,6.2,0.8,200.0,77.0,8.0
25623,129,서산,2021-01-31 21:00,5.3,0.7,230.0,82.0,8.0
25624,129,서산,2021-01-31 22:00,5.8,1.0,200.0,77.0,8.0


In [ ]:
dangjin_obs.isnull().sum()

지점              0
지점명             0
일시              0
기온(°C)         37
풍속(m/s)        36
풍향(16방위)       36
습도(%)          35
전운량(10분위)    3970
dtype: int64

In [ ]:
#예보 시간 컬럼의 데이터 타입을 datetime으로 변경합니다.
dangjin_fcst['Forecast_time'] = pd.to_datetime(dangjin_fcst['Forecast time'])

In [ ]:
# 기상청에서는 하루 3시간 간격으로 8회 예보합니다.
# 여기서는 14시 예보만을 사용하였습니다.
fcst_14 = dangjin_fcst[dangjin_fcst['Forecast_time'].dt.hour==14]

In [ ]:
#다음 날의 기상 예보가 필요하여 예보시간 기준 10시간 후(00:00)부터 33시간 후(23:00) 데이터만 사용합니다.

fcst_14 = fcst_14[(fcst_14['forecast']>=10)&(fcst_14['forecast']<=33)]

In [ ]:
#예보 시점에 focast를 더하여 예보 시각을 구합니다.

def to_date(x):
    return pd.DateOffset(hours=x)

fcst_14['Forecast_time'] = fcst_14['Forecast_time'] + fcst_14['forecast'].map(to_date)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,


In [ ]:
fcst_14 = fcst_14[['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]
fcst_14

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
22,2018-03-02 00:00:00,-2.0,55.0,6.7,336.0,1.0
23,2018-03-02 03:00:00,-4.0,55.0,5.5,339.0,1.0
24,2018-03-02 06:00:00,-5.0,60.0,4.3,344.0,1.0
25,2018-03-02 09:00:00,-2.0,55.0,2.0,345.0,1.0
26,2018-03-02 12:00:00,1.0,45.0,0.8,256.0,1.0
...,...,...,...,...,...,...
162085,2021-03-01 09:00:00,8.0,90.0,3.9,64.0,4.0
162086,2021-03-01 12:00:00,10.0,90.0,4.8,7.0,4.0
162087,2021-03-01 15:00:00,10.0,85.0,6.6,11.0,4.0
162088,2021-03-01 18:00:00,7.0,85.0,6.5,34.0,4.0


In [ ]:
#태양광 발전량 예측은 1시간 간격으로 해야하나 예보는 3시간 간격으로 나옵니다.
# knn & 선형회귀

fcst_14_ = pd.DataFrame()
fcst_14_['Forecast_time'] = pd.date_range(start='2018-03-02 00:00:00', end='2021-03-01 23:00:00', freq='H')

In [ ]:
fcst_14_ = pd.merge(fcst_14_, fcst_14, on='Forecast_time', how='outer')

In [ ]:
fcst_14_

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-02 00:00:00,-2.0,55.0,6.7,336.0,1.0
1,2018-03-02 01:00:00,NaN,NaN,NaN,NaN,NaN
2,2018-03-02 02:00:00,NaN,NaN,NaN,NaN,NaN
3,2018-03-02 03:00:00,-4.0,55.0,5.5,339.0,1.0
4,2018-03-02 04:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,NaN,NaN,NaN,NaN,NaN
26300,2021-03-01 20:00:00,NaN,NaN,NaN,NaN,NaN
26301,2021-03-01 21:00:00,5.0,85.0,5.9,27.0,4.0
26302,2021-03-01 22:00:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=3)

In [ ]:
data_filled_fcst = imputer.fit_transform(fcst_14_.iloc[:,1:])

In [ ]:
fcst_data=pd.DataFrame(data_filled_fcst)

In [ ]:
fcst_data

,0,1,2,3,4
0,-2.000000,55.000000,6.700000,336.000000,1.000000
1,12.538207,77.158075,3.661496,214.181227,2.441264
2,12.538207,77.158075,3.661496,214.181227,2.441264
3,-4.000000,55.000000,5.500000,339.000000,1.000000
4,12.538207,77.158075,3.661496,214.181227,2.441264
...,...,...,...,...,...
26299,12.538207,77.158075,3.661496,214.181227,2.441264
26300,12.538207,77.158075,3.661496,214.181227,2.441264
26301,5.000000,85.000000,5.900000,27.000000,4.000000
26302,12.538207,77.158075,3.661496,214.181227,2.441264


In [ ]:
fcst_filled=pd.concat([fcst_14_['Forecast_time'],fcst_data],axis=1)  


NameError: ignored

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동서발전 태양광 발전량 예측 AI 경진대회/당진/dangjin_fcst_data.csv')
data


,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,52.0,7.0,40.0,3.2,187.0,1.0
162204,2021-03-01 08:00:00,55.0,8.0,40.0,4.5,217.0,1.0
162205,2021-03-01 08:00:00,58.0,5.0,55.0,2.2,210.0,1.0
162206,2021-03-01 08:00:00,61.0,1.0,80.0,1.9,164.0,1.0
